In [1]:
!pip install davos
import davos

In [2]:
smuggle pandas as pd # pip: pandas==1.5.1
smuggle numpy as np # pip: numpy==1.23.4

smuggle os

In [3]:
fname = os.path.join(os.path.split(os.path.split(os.getcwd())[0])[0], 'data', 'raw', 'demographics.csv')
demographics = pd.read_csv(fname)

Add some useful information

In [4]:
demographics['Age'] = demographics['Timestamp'].astype('datetime64[ns]').apply(lambda x: x.year) - demographics['Birth year']
demographics.drop('Birth year', axis=1, inplace=True)

alertness_dict = {'Very alert': 2, 'Fairly alert': 1, 'Neutral': 0, 'A little sluggish': -1, 'Very sluggish': -2}
demographics['Alertness'] = demographics['Alertness'].map(alertness_dict)

# ranges of hours were converted to dates!  correct this...
sleep_map = {'8-Jun': '6 -- 8', '6-Apr': '4 -- 6', '8+': '8+', '0-2': '0 -- 2', '4-Feb': '2 -- 4'}
demographics['Hours of sleep'] = demographics['Hours of sleep'].map(sleep_map)

# numbers of khan courses watched were converted to dates; correct this...
khan_map = {'2-Jan': '1 -- 2', '5-Mar': '3 -- 5', np.nan: 0, '0': '0', '10+': '10+', '10-May': '5 -- 10'}
demographics['Khan courses watched'] = demographics['Khan courses watched'].map(khan_map)

major_category_map = {
    'Business': 'Social sciences',
    'Mathematics and engineering (mathematics, electrical engineering, mechanical engineering, computer science, software engineering, human-computer-interactions, etc.)': 'Mathematics and engineering',
    'BA': 'Undecided',
    'Social sciences (e.g. psychology, sociology, economics, political science, linguistics, anthropology, archaeology, etc.)': 'Social sciences',
    'Natural sciences (e.g. physics, chemistry, biology, neuroscience)': 'Natural sciences',
    'Humanities (history, foreign language, literature, writing, communications, film, etc.)': 'Humanities',
    'Creative arts (music, dance, theater, painting, sculpture, graphic design, etc.)': 'Creative arts',
    'Professional (pre-medical, pre-law, nursing, etc.)': 'Professional',
    'information management': 'Mathematics and engineering',
    'none': 'Undecided',
    'education': 'Social sciences',
    'Undeclared': 'Undecided',
    'Speech Therapy': 'Professional',
    'Business Administration': 'Social sciences',
    'Information Technology': 'Mathematics and engineering',
    None: 'Undecided',
    'undecided': 'Undecided',
    'Currently undecided': 'Undecided'
}
demographics['Undergraduate major (category)'] = demographics['Undergraduate major (category)'].apply(lambda x: ';'.join([major_category_map[m] for m in x.split(';')]))

In [5]:
demographics.head()

,Unnamed: 0,posix_timestamp,Timestamp,Data share consent,Native English,Hearing impairments,Normal color vision,Impaired focus,Gender,Ethnicity,...,Khan Academy user,Khan courses watched,Khan subjects watched,Math courses,Science and engineering courses,Other platform subjects,In-person subjects,Watched Four Fundamental Fources,Watched Birth of Stars,Age
0,0,1.556300e+12,2019/04/26 12:38:19 PM EST,Yes,Yes,No,Yes,No,Female,Not Hispanic or Latino,...,Yes,10+,Math,Algebra 1;Geometry;Algebra 2;Trigonometry;Prec...,None of the above,None of the above,Math;Science & engineering;Arts & humanities;E...,No,No,19
1,1,1.556560e+12,2019/04/29 1:24:02 PM EST,Yes,Yes,No,Yes,No,Female,Not Hispanic or Latino,...,Yes,1 -- 2,Math;Science & engineering,AP Calculus AB,Physics,None of the above,Math;Science & engineering;Arts & humanities,No,No,22
2,2,1.556570e+12,2019/04/29 2:10:09 PM EST,Yes,Yes,No,Yes,No,Male,Not Hispanic or Latino,...,No,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19
3,3,1.556740e+12,2019/05/01 2:12:46 PM EST,Yes,Yes,No,Yes,No,Female,Not Hispanic or Latino,...,Yes,3 -- 5,Math;Science & engineering;Economics & finance,AP Calculus AB;AP Calculus BC,AP Physics 1;Chemistry;AP Chemistry;Biology;Hi...,Economics & finance,None of the above,No,No,20
4,4,1.556900e+12,2019/05/03 12:31:51 PM EST,Yes,Yes,No,Yes,No,Female,Not Hispanic or Latino,...,Yes,10+,Math;Science & engineering;Economics & finance...,Multivariable Calculus;Differential Equations,Chemistry;AP Chemistry;AP Biology,Test prep,Math;Science & engineering;Computing;Arts & hu...,No,No,19


In [6]:
def summarize_counts(df, col):
    print('Raw counts:')
    print(df[col].value_counts())

In [7]:
def summarize_multicounts(df, col):
    counts = {}
    for val in df[col].values:
        try:
            for x in val.split(';'):
                if x in counts:
                    counts[x] += 1
                else:
                    counts[x] = 1
        except:
            pass  # ignore NaNs and empty cells
    
    print('Raw counts:')
    counts = [(key, counts[key]) for key in counts]
    counts.sort(key=lambda x: x[1], reverse=True)

    for k, c in counts:
        print(f'{k}: {c}')

In [8]:
def summarize_vals(df, col):
    return df[col].describe()

# Native English speakers

In [9]:
summarize_counts(demographics, 'Native English')

Raw counts:
Yes    49
No      1
Name: Native English, dtype: int64


# Hearing impairments

In [10]:
summarize_counts(demographics, 'Hearing impairments')

Raw counts:
No     49
Yes     1
Name: Hearing impairments, dtype: int64


# Normal color vision

In [11]:
summarize_counts(demographics, 'Normal color vision')

Raw counts:
Yes    49
No      1
Name: Normal color vision, dtype: int64


# Impaired focus

In [12]:
summarize_counts(demographics, 'Impaired focus')

Raw counts:
No    50
Name: Impaired focus, dtype: int64


# Age

In [13]:
summarize_vals(demographics, 'Age')

count    50.00000
mean     19.52000
std       1.09246
min      18.00000
25%      19.00000
50%      19.00000
75%      20.00000
max      22.00000
Name: Age, dtype: float64

# Gender

In [14]:
summarize_counts(demographics, 'Gender')

Raw counts:
Female    35
Male      15
Name: Gender, dtype: int64


# Ethnicity

In [15]:
summarize_counts(demographics, 'Ethnicity')

Raw counts:
Not Hispanic or Latino    47
Hispanic or Latino         3
Name: Ethnicity, dtype: int64


# Race

In [16]:
summarize_multicounts(demographics, 'Race')

Raw counts:
White: 31
Asian: 14
Black or African American: 5
A mix: Indian and White: 1
Native Hawaiian or Other Pacific Islander: 1
Prefer not to say: 1


# Highest degree achieved

In [17]:
summarize_counts(demographics, 'Highest degree')

Raw counts:
Some college            33
High school graduate    16
Bachelor's degree        1
Name: Highest degree, dtype: int64


# Undergraduate major (category)

In [18]:
summarize_multicounts(demographics, 'Undergraduate major (category)')

Raw counts:
Social sciences: 28
Natural sciences: 16
Professional: 8
Mathematics and engineering: 7
Humanities: 4
Undecided: 3


# Sleep

In [19]:
summarize_counts(demographics, 'Hours of sleep')

Raw counts:
6 -- 8    35
4 -- 6     9
8+         5
2 -- 4     1
Name: Hours of sleep, dtype: int64


# Coffee consumption

In [20]:
summarize_counts(demographics, 'Cups of coffee')

Raw counts:
0     38
1     10
4+     1
3      1
Name: Cups of coffee, dtype: int64


# Alertness

In [21]:
summarize_vals(demographics, 'Alertness')

count    50.000000
mean     -0.100000
std       0.839096
min      -2.000000
25%      -1.000000
50%       0.000000
75%       1.000000
max       1.000000
Name: Alertness, dtype: float64

# Khan academy exposure

In [22]:
# ever used khan academy?
summarize_counts(demographics, 'Khan Academy user')

Raw counts:
Yes    46
No      4
Name: Khan Academy user, dtype: int64


In [23]:
# number of khan courses watched
summarize_counts(demographics, 'Khan courses watched')

Raw counts:
10+        19
3 -- 5     11
5 -- 10     8
1 -- 2      7
0           4
0           1
Name: Khan courses watched, dtype: int64


In [24]:
# khan math subjects watched
summarize_multicounts(demographics, 'Math courses')

Raw counts:
AP Calculus AB: 21
Precalculus: 17
Algebra 2: 14
AP Calculus BC: 12
Trigonometry: 11
Algebra 1: 10
Geometry: 8
Pre-algebra: 7
None of the above: 6
Multivariable Calculus: 5
Differential Equations: 5
Statistics & Probability: 4
AP Statistics: 2
Linear Algebra: 2
Early Math: 1
Arithmetic: 1


In [25]:
# khan science and engineering courses watched
summarize_multicounts(demographics, 'Science and engineering courses')

Raw counts:
None of the above: 20
Chemistry: 12
Physics: 10
AP Chemistry: 8
AP Biology: 7
AP Physics 1: 5
Biology: 5
High school Biology: 3
AP Physics 2: 3
Organic Chemistry: 1
Health & Medicine: 1


In [26]:
# Watched Four Fundemantal Fources?
summarize_counts(demographics, 'Watched Four Fundamental Fources')

Raw counts:
No          45
Not sure     1
Name: Watched Four Fundamental Fources, dtype: int64


In [27]:
# Watched Birth of Stars?
summarize_counts(demographics, 'Watched Birth of Stars')

Raw counts:
No    46
Name: Watched Birth of Stars, dtype: int64


# Non-Khan academy courses (online)

In [28]:
summarize_multicounts(demographics, 'Other platform subjects')

Raw counts:
None of the above: 18
Math: 15
Science & engineering: 11
Test prep: 9
Economics & finance: 3
Arts & humanities: 2
Computing: 2


# In-person courses

In [29]:
summarize_multicounts(demographics, 'In-person subjects')

Raw counts:
Math: 39
Science & engineering: 38
Arts & humanities: 35
Test prep: 27
Economics & finance: 26
Computing: 15
College, careers, & more: 7
None of the above: 6
